In [235]:
import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from scipy import interpolate
from scipy.signal import savgol_filter

#Load file into 2d list
with open('../data/P7132033_37.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

#Remove NA values
#zc_str[:] = [x for x in zc_str if not x[1] == 'NA']


#Format zc_str to floats
zc_noisy = [[float(point[0]), float(point[1])] for point in zc_str]

#Remove cutoff values
zc_noisy[:] = [x for x in zc_noisy if x[1] > 44000]

#Get dy, dy2, and x
prev_point = [0, 0]
dy = list()
dy2 = list()
x_noisy = list()
for point in zc_noisy:
    dy.append(numpy.sqrt((point[0]-prev_point[0])**2 + (point[1]-prev_point[1])**2))
    if not len(dy) == 1:
        dy2.append(numpy.sqrt((point[0]-prev_point[0])**2 + (dy[-1]-dy[-2])**2))
    x_noisy.append(point[0])
    prev_point = point
    
#Define cutoff dy for ignoring noise
cutoff = 1000

#Smooth random spikes
i = 2
#while i < len(dy) - 1:
    #if dy[i - 1] < cutoff and dy[i] > cutoff and dy[i + 1] < cutoff:
        #dy[i] = 0
    #i += 1
            
#ELIMINATE NOISE
i = 0
zc = list()
while i < len(dy):
    if dy[i] < cutoff:
        zc.append(zc_noisy[i])
    i += 1
    
#Define pulses
i = 0
pulses = list()
while i < len(zc):
    if dy[i] < cutoff:
        pulses.append([zc[i][0], zc[i][1]])
    i+=1
    
yhat2 = savgol_filter([point[1] for point in zc_noisy], 27, 2) # window size 51, polynomial order 3
yhat3 = savgol_filter([point[1] for point in zc_noisy], 17, 3) # window size 51, polynomial order 3

#Graph ZC data and dy data
trace_noisy = graphs.Scatter(
                                x = [point[0] for point in zc_noisy], 
                                y = [point[1] for point in zc_noisy],
                                mode = 'markers',
                                name = 'ZC- noisy'
                            )
trace_smooth = graphs.Scatter(
                            x = [point[0] for point in zc_noisy], 
                            y = yhat2,
                            name = 'ZC- clean'
                         )
trace_dy = graphs.Scatter(
                            x = x_noisy, 
                            y = dy2, 
                            name = 'Derivative'
                         )
trace_pulses = graphs.Scatter(
                                x = [point[0] for point in pulses], 
                                y = [point[1] for point in pulses], 
                                name = 'Pulses'
                             )
trace_cutoff = graphs.Scatter(
                                x = [x_noisy[0], x_noisy[-1]], 
                                y = [cutoff, cutoff], 
                                name = 'Cutoff'
                             )
trace = [trace_noisy, trace_smooth, trace_dy, trace_cutoff]
py.iplot(trace, filename='P7132033_37')